In [3]:
import pandas as pd
import requests

# Function to calculate distance between two points using Mapbox Directions API
def calculate_distance(lat1, lon1, lat2, lon2, access_token):
    url = f"https://api.mapbox.com/directions/v5/mapbox/driving/{lon1},{lat1};{lon2},{lat2}?access_token={access_token}"
    response = requests.get(url)
    data = response.json()
    if 'routes' in data:
        distance = data['routes'][0]['distance'] / 1000  # Distance in kilometers
        return distance
    else:
        return None

# Load datasets
buildings_data = pd.read_csv("buildings_dataset.csv")
neighborhood_data = pd.read_csv("neighborhood.csv")

# Mapbox access token
mapbox_access_token = "pk.eyJ1Ijoic2FuaWthZCIsImEiOiJjbHUybGxvbnUwNDVnMmpuNG43bWw3NWxvIn0.2IgFosltIpszsqhwMvQZAw"

# Prompt the user to select two neighborhood priorities
print("Select two preferred neighborhood priorities:")
print("1. School")
print("2. Hospital")
print("3. Railway Station")
print("4. Metro Station")

# Mapping of priority input to neighborhood type
priority_mapping = {
    1: "School",
    2: "Hospital",
    3: "Railway Station",
    4: "Metro Station"
}

# Input for the first priority
priority_input_1 = int(input("Enter the number corresponding to the first preferred priority: "))
if priority_input_1 not in priority_mapping:
    print("Invalid input. Please enter a number between 1 and 4.")
    exit()

# Input for the second priority
priority_input_2 = int(input("Enter the number corresponding to the second preferred priority: "))
if priority_input_2 not in priority_mapping:
    print("Invalid input. Please enter a number between 1 and 4.")
    exit()

# Selected priorities
selected_priority_1 = priority_mapping[priority_input_1]
selected_priority_2 = priority_mapping[priority_input_2]
print("Your selected priorities are:", selected_priority_1, "and", selected_priority_2)

# Filter neighborhood data based on selected priorities
filtered_neighborhood_data_1 = neighborhood_data[neighborhood_data['n_type'] == selected_priority_1]
filtered_neighborhood_data_2 = neighborhood_data[neighborhood_data['n_type'] == selected_priority_2]

# Calculate distances between buildings and each selected priority using Mapbox Directions API
combined_data = buildings_data.copy()
combined_data['Nearest_Priority_1_Dist'] = combined_data.apply(lambda row: min([calculate_distance(row['Latitude'], row['Longitude'], n_row['Latitude'], n_row['Longitude'], mapbox_access_token) for _, n_row in filtered_neighborhood_data_1.iterrows()]), axis=1)
combined_data['Nearest_Priority_2_Dist'] = combined_data.apply(lambda row: min([calculate_distance(row['Latitude'], row['Longitude'], n_row['Latitude'], n_row['Longitude'], mapbox_access_token) for _, n_row in filtered_neighborhood_data_2.iterrows()]), axis=1)

# Print top 10 recommended buildings
print("Top 10 Recommended Buildings:")
top_buildings = combined_data.sort_values(by='Nearest_Priority_1_Dist').head(10)
for index, row in top_buildings.iterrows():
    n_name_1 = filtered_neighborhood_data_1.iloc[int(row['Nearest_Priority_1_Dist'])]['n_name']
    n_name_2 = filtered_neighborhood_data_2.iloc[int(row['Nearest_Priority_2_Dist'])]['n_name']
    print(f"Building ID: {row['prop_id']}, Name: {row['prop_name']}")
    print(f"Nearest {selected_priority_1}: {n_name_1}, Distance: {row['Nearest_Priority_1_Dist']:.2f} km")
    print(f"Nearest {selected_priority_2}: {n_name_2}, Distance: {row['Nearest_Priority_2_Dist']:.2f} km")
    print()



Select two preferred neighborhood priorities:
1. School
2. Hospital
3. Railway Station
4. Metro Station
Your selected priorities are: School and Hospital
Top 10 Recommended Buildings:
Building ID: 3, Name: Sonali Apartment
Nearest School: Garodia International Centre for Learning Mumbai, Distance: 0.11 km
Nearest Hospital: Parakh Hospital
, Distance: 0.25 km

Building ID: 79, Name: Sonali Apartment
Nearest School: Garodia International Centre for Learning Mumbai, Distance: 0.11 km
Nearest Hospital: Parakh Hospital
, Distance: 0.25 km

Building ID: 55, Name: Sonali Apartment
Nearest School: Garodia International Centre for Learning Mumbai, Distance: 0.11 km
Nearest Hospital: Parakh Hospital
, Distance: 0.25 km

Building ID: 35, Name: Anant Chhaya
Nearest School: Garodia International Centre for Learning Mumbai, Distance: 0.12 km
Nearest Hospital: Parakh Hospital
, Distance: 0.32 km

Building ID: 66, Name: Anant Chhaya
Nearest School: Garodia International Centre for Learning Mumbai, Dis

In [1]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from geopy.distance import geodesic

# Loading datasets
buildings_data = pd.read_csv("buildings_dataset.csv")
neighborhood_data = pd.read_csv("neighborhood.csv")

# Select their preferred neighborhood priority
print("Select your preferred neighborhood priority:")
print("1. School")
print("2. Hospital")
print("3. Railway Station")
print("4. Metro Station")

priority_mapping = {
    1: "School",
    2: "Hospital",
    3: "Railway Station",
    4: "Metro Station"
}

priority_input = int(input("Enter the number corresponding to your preferred priority: "))

# Validate input
if priority_input not in priority_mapping:
    print("Invalid input. Please enter a number between 1 and 4.")
else:
    selected_priority = priority_mapping[priority_input]
    print("Your selected priority is:", selected_priority)

    # Filtering neighborhood.csv based on selected priority
    filtered_neighborhood_data = neighborhood_data[neighborhood_data['n_type'] == selected_priority]

    # Preprocessing buildings_dataset data
    X_buildings = buildings_data[['Latitude', 'Longitude']]

    # Preprocessing neighborhood.csv
    X_neighborhood = filtered_neighborhood_data[['Latitude', 'Longitude']]

    # KNN
    knn = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(X_neighborhood)
    distances, indices = knn.kneighbors(X_buildings)

    # Calculating distances in kilometers
    distances_km = [geodesic((buildings_data.loc[i, 'Latitude'], buildings_data.loc[i, 'Longitude']),
                             (filtered_neighborhood_data.loc[indices[i][0], 'Latitude'], filtered_neighborhood_data.loc[indices[i][0], 'Longitude'])).kilometers
                    for i in range(len(buildings_data))]

    # Add nearest neighborhood information and distance to buildings dataset
    buildings_data['Nearest_Neighborhood_ID'] = filtered_neighborhood_data.iloc[indices.flatten()]['n_id'].values
    buildings_data['Distance_to_Nearest_Neighborhood'] = distances_km

    # Sort buildings by distance
    sorted_buildings_data = buildings_data.sort_values(by='Distance_to_Nearest_Neighborhood')

    # Print top 10 recommended buildings
    print("\n")
    print("Top 10 Recommended Buildings:")
    print("\n")
    for index, row in sorted_buildings_data.head(10).iterrows():
        nearest_neighborhood_name = filtered_neighborhood_data[filtered_neighborhood_data['n_id'] == row['Nearest_Neighborhood_ID']]['n_name'].iloc[0]
        print(f"Building ID: {row['prop_id']}, Name: {row['prop_name']}")
        print(f"Nearest {selected_priority}: {row['Nearest_Neighborhood_ID']}, {nearest_neighborhood_name}")
        print(f"Distance: {row['Distance_to_Nearest_Neighborhood']:.2f} km")
        print()


ModuleNotFoundError: No module named 'sklearn'

In [1]:
import pandas as pd
import requests

# Function to calculate distance between two points using Mapbox Directions API
def calculate_distance(lat1, lon1, lat2, lon2, access_token):
    url = f"https://api.mapbox.com/directions/v5/mapbox/driving/{lon1},{lat1};{lon2},{lat2}?access_token={access_token}"
    response = requests.get(url)
    data = response.json()
    if 'routes' in data:
        distance = data['routes'][0]['distance'] / 1000  # Distance in kilometers
        return distance
    else:
        return None

# Load datasets
buildings_data = pd.read_csv("25_buildings.csv")
neighborhood_data = pd.read_csv("neighborhood.csv")

# Mapbox access token
mapbox_access_token = "pk.eyJ1Ijoic2FuaWthZCIsImEiOiJjbHUybGxvbnUwNDVnMmpuNG43bWw3NWxvIn0.2IgFosltIpszsqhwMvQZAw"

# Prompt the user to select priorities
num_priorities = int(input("Enter the number of priorities: "))
priorities = {}
for i in range(num_priorities):
    print(f"Select priority {i+1}:")
    print("1. School")
    print("2. Hospital")
    print("3. Railway Station")
    print("4. Metro Station")
    priority_input = int(input("Enter the number corresponding to the priority: "))
    if priority_input not in range(1, 5):
        print("Invalid input. Please enter a number between 1 and 4.")
        exit()
    priorities[i] = priority_input

# Mapping of priority input to neighborhood type
priority_mapping = {
    1: "School",
    2: "Hospital",
    3: "Railway Station",
    4: "Metro Station"
}

# Filter neighborhood data based on selected priorities
filtered_neighborhood_data = {}
for i, priority_input in priorities.items():
    selected_priority = priority_mapping[priority_input]
    filtered_neighborhood_data[i] = neighborhood_data[neighborhood_data['n_type'] == selected_priority]

# Calculate distances between buildings and each selected priority using Mapbox Directions API
combined_data = buildings_data.copy()
for i, priority_input in priorities.items():
    combined_data[f'Nearest_Priority_{i+1}_Dist'] = combined_data.apply(lambda row: min([calculate_distance(row['Latitude'], row['Longitude'], n_row['Latitude'], n_row['Longitude'], mapbox_access_token) for _, n_row in filtered_neighborhood_data[i].iterrows()]), axis=1)

# Print top 10 recommended buildings
print("Top 10 Recommended Buildings:")
top_buildings = combined_data.sort_values(by=['Nearest_Priority_1_Dist', 'Nearest_Priority_2_Dist']).head(10)
for index, row in top_buildings.iterrows():
    nearest_priorities = []
    for i, priority_input in priorities.items():
        n_name = filtered_neighborhood_data[i].iloc[int(row[f'Nearest_Priority_{i+1}_Dist'])]['n_name']
        nearest_priorities.append((priority_mapping[priority_input], n_name, row[f'Nearest_Priority_{i+1}_Dist']))
    print(f"Building ID: {row['prop_id']}, Name: {row['prop_name']}")
    for priority, n_name, distance in nearest_priorities:
        print(f"Nearest {priority}: {n_name}, Distance: {distance:.2f} km")
    print()


Select priority 1:
1. School
2. Hospital
3. Railway Station
4. Metro Station
Select priority 2:
1. School
2. Hospital
3. Railway Station
4. Metro Station
Select priority 3:
1. School
2. Hospital
3. Railway Station
4. Metro Station


In [4]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import requests
import json

# Define the URL and headers for the API request
url = "https://api.mapbox.com/directions-matrix/v1/mapbox/driving/2.344003,48.85805;2.34675,48.85727"
headers = {
    'Content-Type': 'application/json'
}

response = requests.post(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print(f"Request failed with status code {response.status_code}")


# Load datasets
buildings_data = pd.read_csv("25_buildings.csv")
neighborhood_data = pd.read_csv("neighborhood.csv")

# User Input and Priority Selection
num_priorities = int(input("Enter the number of priorities: "))
priorities = {}
for i in range(num_priorities):
    print(f"Select priority {i+1}:")
    print("1. School")
    print("2. Hospital")
    print("3. Railway Station")
    print("4. Metro Station")
    priority_input = int(input("Enter the number corresponding to the priority: "))
    if priority_input not in range(1, 5):
        print("Invalid input. Please enter a number between 1 and 4.")
        exit()
    priorities[i] = priority_input

# Mapping of priority input to neighborhood type
priority_mapping = {
    1: "School",
    2: "Hospital",
    3: "Railway Station",
    4: "Metro Station"
}

# Preprocessing buildings data
X_buildings = buildings_data[['Latitude', 'Longitude']]

# Preprocessing neighborhood data
X_neighborhoods = neighborhood_data[['Latitude', 'Longitude']]

# Assuming the nearest_neighbors_indices is a list of lists
suggested_buildings = buildings_data.copy()
for j in range(len(buildings_data)):
    if j < len(nearest_neighbors_indices):  # Check if the index is within the range of nearest_neighbors_indices
        distances_km = []
        for index in nearest_neighbors_indices[j]:
            if index < len(neighborhood_data):
                building_coords = (suggested_buildings.loc[j, 'Latitude'], suggested_buildings.loc[j, 'Longitude'])
                neighborhood_coords = (neighborhood_data.iloc[index]['Latitude'], neighborhood_data.iloc[index]['Longitude'])
                distance = get_distance(building_coords, neighborhood_coords)
                if distance is not None:
                    distances_km.append(distance)
                else:
                    distances_km.append(float('inf'))  # Use a large number to represent infinite distance
            else:
                print(f"Invalid index: {index}")
        suggested_buildings[f'Nearest_Priority_{i+1}_Dist'] = distances_km
    else:
        print(f"Index {j} out of range for nearest_neighbors_indices")

# (Remaining code)

# KNN to find nearest neighbors
nearest_neighbors_indices = []
for i in range(num_priorities):
    knn = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(X_neighborhoods)
    _, indices = knn.kneighbors(X_buildings)
    nearest_neighbors_indices.append(indices.flatten())

# Function to calculate distance using Mapbox API
def get_distance(coords1, coords2):
    access_token = "pk.eyJ1Ijoic2FuaWthZCIsImEiOiJjbHVrNGdmcjgwZnB0MmpsNXUwdmcyMDR6In0.M4cFaJUCdMnCT0plVTnqVQ"  # Replace with your Mapbox access token
    url = f"https://api.mapbox.com/directions-matrix/v1/mapbox/driving/{coords1[0]},{coords1[1]};{coords2[0]},{coords2[1]}"
    params = {
        'access_token': access_token
    }

    try:
        response = requests.post(url, headers=headers, params=params)
        response.raise_for_status()  # Raise an HTTPError for bad status codes
        data = response.json()
        # Process the API response and return the distance
        # (Remaining code)
    except requests.exceptions.RequestException as e:
        print(f"Error occurred during API request: {e}")
        return None

# Filter buildings within 3 kilometers of any priority
suggested_buildings = buildings_data.copy()
for i in range(num_priorities):
    distances_km = []
    for j in range(len(buildings_data)):
        for index in nearest_neighbors_indices[j]:
            if index < len(neighborhood_data):
                building_coords = (suggested_buildings.loc[j, 'Latitude'], suggested_buildings.loc[j, 'Longitude'])
                neighborhood_coords = (neighborhood_data.iloc[index]['Latitude'], neighborhood_data.iloc[index]['Longitude'])
                distance = get_distance(building_coords, neighborhood_coords)
                if distance is not None:
                    distances_km.append(distance)
                else:
                    distances_km.append(float('inf'))  # Use a large number to represent infinite distance
            else:
                print(f"Invalid index: {index}")
    suggested_buildings[f'Nearest_Priority_{i+1}_Dist'] = distances_km

# (Remaining code)


# Print top 5 recommended buildings within 3 kilometers
if len(suggested_buildings) > 0:
    print("Top 5 Recommended Buildings within 3 kilometers:")
    for index, row in suggested_buildings.head(5).iterrows():
        if index < len(suggested_buildings):
            print(f"Building ID: {row['prop_id']}, Name: {row['prop_name']}")
            for i in range(num_priorities):
                if row[f'Nearest_Priority_{i+1}_Dist'] is not None and row[f'Nearest_Priority_{i+1}_Dist'] <= 3:
                    if nearest_neighbors_indices[i][index] < len(neighborhood_data):
                        n_name = neighborhood_data.iloc[nearest_neighbors_indices[i][index]]['n_name']
                        print(f"Nearest {priority_mapping[priorities[i]]}: {n_name}, Distance: {row[f'Nearest_Priority_{i+1}_Dist']:.2f} km")
            print()
else:
    print("No results")



Request failed with status code 401
Select priority 1:
1. School
2. Hospital
3. Railway Station
4. Metro Station
Select priority 2:
1. School
2. Hospital
3. Railway Station
4. Metro Station
Error occurred during API request: 422 Client Error: Unknown for url: https://api.mapbox.com/directions-matrix/v1/mapbox/driving/72.9113553,19.0867632;72.9104529,19.0853873?sources=0&destinations=1&access_token=pk.eyJ1Ijoic2FuaWthZCIsImEiOiJjbHVrNGdmcjgwZnB0MmpsNXUwdmcyMDR6In0.M4cFaJUCdMnCT0plVTnqVQ
Error occurred during API request: 422 Client Error: Unknown for url: https://api.mapbox.com/directions-matrix/v1/mapbox/driving/72.9113553,19.0867632;72.9096172,19.0874846?sources=0&destinations=1&access_token=pk.eyJ1Ijoic2FuaWthZCIsImEiOiJjbHVrNGdmcjgwZnB0MmpsNXUwdmcyMDR6In0.M4cFaJUCdMnCT0plVTnqVQ
Error occurred during API request: 422 Client Error: Unknown for url: https://api.mapbox.com/directions-matrix/v1/mapbox/driving/72.9113553,19.0867632;72.9140609,19.0858279?sources=0&destinations=1&access_tok

IndexError: list index out of range

In [7]:
# SINGLE ONE
import pandas as pd
from sklearn.neighbors import NearestNeighbors  # Add this import statement
import requests
from geopy.distance import geodesic

# Define the get_distance function
def get_distance(coords1, coords2):
    access_token = "pk.eyJ1Ijoic2FuaWthZCIsImEiOiJjbHVrNGdmcjgwZnB0MmpsNXUwdmcyMDR6In0.M4cFaJUCdMnCT0plVTnqVQ"  # Replace with your Mapbox access token
    url = f"https://api.mapbox.com/directions-matrix/v1/mapbox/driving/{coords1[1]},{coords1[0]};{coords2[1]},{coords2[0]}"
    params = {
        'access_token': access_token
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an HTTPError for bad status codes
        data = response.json()
        distance_meters = data['durations'][0][1]  # Distance in meters
        distance_km = distance_meters / 1000  # Convert meters to kilometers
        return distance_km
    except requests.exceptions.RequestException as e:
        print(f"Error occurred during API request: {e}")
        return None
    except (KeyError, IndexError) as e:
        print(f"Invalid data format in the API response: {e}")
        return None


# Load datasets
buildings_data = pd.read_csv("25_buildings.csv")
neighborhood_data = pd.read_csv("neighborhood.csv")

# Select the preferred neighborhood priority
print("Select your preferred neighborhood priority:")
print("1. School")
print("2. Hospital")
print("3. Railway Station")
print("4. Metro Station")

priority_mapping = {
    1: "School",
    2: "Hospital",
    3: "Railway Station",
    4: "Metro Station"
}

priority_input = int(input("Enter the number corresponding to your preferred priority: "))

# Validate input
if priority_input not in priority_mapping:
    print("Invalid input. Please enter a number between 1 and 4.")
else:
    selected_priority = priority_mapping[priority_input]
    print("Your selected priority is:", selected_priority)

    # Filtering neighborhood.csv based on selected priority
    filtered_neighborhood_data = filtered_neighborhood_data.reset_index(drop=True)
    filtered_neighborhood_data = neighborhood_data[neighborhood_data['n_type'] == selected_priority]

    # Preprocessing buildings_dataset data
    X_buildings = buildings_data[['Latitude', 'Longitude']]

    # Preprocessing neighborhood.csv
    X_neighborhood = filtered_neighborhood_data[['Latitude', 'Longitude']]

    # KNN to find nearest neighbors
    knn = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(X_neighborhood)
    _, indices = knn.kneighbors(X_buildings)

    # Calculate distances using Mapbox API
    # Calculating distances using Mapbox API
    distances_km = []
    for i in range(len(buildings_data)):
        coords1 = (buildings_data.loc[i, 'Latitude'], buildings_data.loc[i, 'Longitude'])
        nearest_neighbor_index = indices[i][0]  # Index of the nearest neighbor in X_neighborhood
        nearest_neighbor_coords = (X_neighborhood.iloc[nearest_neighbor_index]['Latitude'], X_neighborhood.iloc[nearest_neighbor_index]['Longitude'])
        distance_km = get_distance(coords1, nearest_neighbor_coords)  # Using Mapbox API to calculate road distance
        distances_km.append(distance_km)


    # Add nearest neighborhood information and distance to buildings dataset
    buildings_data['Nearest_Neighborhood_ID'] = filtered_neighborhood_data.iloc[indices.flatten()]['n_id'].values
    buildings_data['Distance_to_Nearest_Neighborhood'] = distances_km

    # Sort buildings by distance
    sorted_buildings_data = buildings_data.sort_values(by='Distance_to_Nearest_Neighborhood')

    # Print top 5 recommended buildings
    if len(sorted_buildings_data) > 0:
        print("\nTop 5 Recommended Buildings:")
        for index, row in sorted_buildings_data.head(5).iterrows():
            nearest_neighborhood_name = filtered_neighborhood_data[
                filtered_neighborhood_data['n_id'] == row['Nearest_Neighborhood_ID']]['n_name'].iloc[0]
            print(f"Building ID: {row['prop_id']}, Name: {row['prop_name']}")
            print(f"Nearest {selected_priority}: {row['Nearest_Neighborhood_ID']}, {nearest_neighborhood_name}")
            print(f"Distance: {row['Distance_to_Nearest_Neighborhood']:.2f} km")
            print()
    else:
        print("No results")

# Function to calculate distance using Mapbox API with enhanced error handling
def get_distance(coords1, coords2):
    access_token = "pk.eyJ1Ijoic2FuaWthZCIsImEiOiJjbHVrNGdmcjgwZnB0MmpsNXUwdmcyMDR6In0.M4cFaJUCdMnCT0plVTnqVQ"  # Replace with your Mapbox access token
    url = f"https://api.mapbox.com/directions-matrix/v1/mapbox/driving/{coords1[1]},{coords1[0]};{coords2[1]},{coords2[0]}"
    params = {
        'access_token': access_token
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an HTTPError for bad status codes
        data = response.json()
        distance_meters = data['durations'][0][1]  # Distance in meters
        distance_km = distance_meters / 1000  # Convert meters to kilometers
        return distance_km
    except requests.exceptions.RequestException as e:
        print(f"Error occurred during API request: {e}")
        return None
    except (KeyError, IndexError) as e:
        print(f"Invalid data format in the API response: {e}")
        return None


Select your preferred neighborhood priority:
1. School
2. Hospital
3. Railway Station
4. Metro Station


Your selected priority is: Hospital

Top 5 Recommended Buildings:
Building ID: 13, Name: Drushti Emerald
Nearest Hospital: 48, Khus-Hal Hospital

Distance: 0.04 km

Building ID: 23, Name: Drushti Emerald
Nearest Hospital: 48, Khus-Hal Hospital

Distance: 0.04 km

Building ID: 20, Name: Kesar Scion
Nearest Hospital: 48, Khus-Hal Hospital

Distance: 0.05 km

Building ID: 1, Name: Kesar Scion
Nearest Hospital: 48, Khus-Hal Hospital

Distance: 0.05 km

Building ID: 10, Name: Aryana Heights
Nearest Hospital: 48, Khus-Hal Hospital

Distance: 0.07 km



In [1]:
#FINAL ONE FOR SINGLE PRIORITY
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import requests

# Define the get_distance function
def get_distance(coords1, coords2):
    access_token = "pk.eyJ1Ijoic2FuaWthZCIsImEiOiJjbHVrNGdmcjgwZnB0MmpsNXUwdmcyMDR6In0.M4cFaJUCdMnCT0plVTnqVQ"  # Replace with your Mapbox access token
    url = f"https://api.mapbox.com/directions-matrix/v1/mapbox/driving/{coords1[1]},{coords1[0]};{coords2[1]},{coords2[0]}"
    params = {
        'access_token': access_token
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an HTTPError for bad status codes
        data = response.json()
        distance_meters = data['durations'][0][1]  # Distance in meters
        distance_km = distance_meters / 1000  # Convert meters to kilometers
        return distance_km
    except requests.exceptions.RequestException as e:
        print(f"Error occurred during API request: {e}")
        return None
    except (KeyError, IndexError) as e:
        print(f"Invalid data format in the API response: {e}")
        return None

# Load datasets
buildings_data = pd.read_csv("25_buildings.csv")
neighborhood_data = pd.read_csv("neighborhood.csv")

# Select the preferred neighborhood priority
print("Select your preferred neighborhood priority:")
print("1. School")
print("2. Hospital")
print("3. Railway Station")
print("4. Metro Station")

priority_mapping = {
    1: "School",
    2: "Hospital",
    3: "Railway Station",
    4: "Metro Station"
}

priority_input = int(input("Enter the number corresponding to your preferred priority: "))

# Validate input
if priority_input not in priority_mapping:
    print("Invalid input. Please enter a number between 1 and 4.")
else:
    selected_priority = priority_mapping[priority_input]
    print("Your selected priority is:", selected_priority)

    # Filtering neighborhood.csv based on selected priority
    filtered_neighborhood_data = neighborhood_data[neighborhood_data['n_type'] == selected_priority].reset_index(drop=True)

    # Preprocessing buildings_dataset data
    X_buildings = buildings_data[['Latitude', 'Longitude']]

    # Preprocessing neighborhood.csv
    X_neighborhood = filtered_neighborhood_data[['Latitude', 'Longitude']]

    # KNN to find nearest neighbors
    knn = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(X_neighborhood)
    _, indices = knn.kneighbors(X_buildings)

    # Calculate distances using Mapbox API
    distances_km = []
    for i in range(len(buildings_data)):
        coords1 = (buildings_data.loc[i, 'Latitude'], buildings_data.loc[i, 'Longitude'])
        coords2 = (filtered_neighborhood_data.loc[indices[i][0], 'Latitude'], filtered_neighborhood_data.loc[indices[i][0], 'Longitude'])
        distance_km = get_distance(coords1, coords2)  # Using Mapbox API to calculate road distance
        distances_km.append(distance_km)

    # Add nearest neighborhood information and distance to buildings dataset
    buildings_data['Nearest_Neighborhood_ID'] = filtered_neighborhood_data.iloc[indices.flatten()]['n_id'].values
    buildings_data['Distance_to_Nearest_Neighborhood'] = distances_km

    # Sort buildings by distance
    sorted_buildings_data = buildings_data.sort_values(by='Distance_to_Nearest_Neighborhood')

    # Print top 5 recommended buildings
    if len(sorted_buildings_data) > 0:
        print("\nTop 5 Recommended Buildings:")
        for index, row in sorted_buildings_data.head(5).iterrows():
            nearest_neighborhood_name = filtered_neighborhood_data[
                filtered_neighborhood_data['n_id'] == row['Nearest_Neighborhood_ID']]['n_name'].iloc[0]
            print(f"Building ID: {row['prop_id']}, Name: {row['prop_name']}")
            print(f"Nearest {selected_priority}: {row['Nearest_Neighborhood_ID']}, {nearest_neighborhood_name}")
            # print(f"Distance: {row['Distance_to_Nearest_Neighborhood']:.2f} km")
            print()
    else:
        print("No results")


Select your preferred neighborhood priority:
1. School
2. Hospital
3. Railway Station
4. Metro Station
Your selected priority is: Hospital

Top 5 Recommended Buildings:
Building ID: 13, Name: Drushti Emerald
Nearest Hospital: 48, Khus-Hal Hospital


Building ID: 23, Name: Drushti Emerald
Nearest Hospital: 48, Khus-Hal Hospital


Building ID: 20, Name: Kesar Scion
Nearest Hospital: 48, Khus-Hal Hospital


Building ID: 1, Name: Kesar Scion
Nearest Hospital: 48, Khus-Hal Hospital


Building ID: 10, Name: Aryana Heights
Nearest Hospital: 48, Khus-Hal Hospital




In [3]:
import pandas as pd
import requests
from sklearn.neighbors import NearestNeighbors

# Function to calculate distance using Mapbox API
def get_distance_from_mapbox(origin, destination, access_token):
    url = f"https://api.mapbox.com/directions/v5/mapbox/driving/{origin[0]},{origin[1]};{destination[0]},{destination[1]}"
    params = {
        "access_token": access_token,
    }
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        distance = data["routes"][0]["distance"]  # Extract the distance from the API response
        return distance
    else:
        return None

# Load building and neighborhood data
buildings_data = pd.read_csv("25_buildings.csv")
neighborhood_data = pd.read_csv("neighborhood.csv")

# User input and priority selection
selected_priorities = ["Railway Station", "Hospital", "School"]

# Calculate distance from each building to all selected priorities using Mapbox API
access_token = "pk.eyJ1Ijoic2FuaWthZCIsImEiOiJjbHVrNGdmcjgwZnB0MmpsNXUwdmcyMDR6In0.M4cFaJUCdMnCT0plVTnqVQ"  # Replace with your Mapbox access token
for priority in selected_priorities:
    destination = neighborhood_data[neighborhood_data['type'] == priority][['latitude', 'longitude']].values[0]
    buildings_data[f'Distance_to_{priority}'] = buildings_data.apply(lambda row: get_distance_from_mapbox((row['latitude'], row['longitude']), destination, access_token), axis=1)

# Use scikit-learn's KNN to find the nearest building to each priority
X = buildings_data[[f'Distance_to_{priority}' for priority in selected_priorities]].values
knn = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(X)

# Find the nearest building for each priority
for priority in selected_priorities:
    distances = buildings_data[f'Distance_to_{priority}'].values.reshape(-1, 1)
    _, indices = knn.kneighbors(distances)
    buildings_data[f'Nearest_to_{priority}'] = buildings_data.iloc[indices.flatten()][['Building ID', 'Name']]

# Display the results
print("Building distances to selected priorities:")
print(buildings_data[['Building ID', 'Name'] + [f'Distance_to_{priority}' for priority in selected_priorities]])
print("\nNearest building to each priority:")
for priority in selected_priorities:
    print(f"{priority}: {buildings_data[f'Nearest_to_{priority}'].values[0]}")


KeyError: 'type'

SO tis is my code which has many errors:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import requests

# Load datasets
buildings_data = pd.read_csv("25_buildings.csv")
neighborhood_data = pd.read_csv("neighborhood.csv")

# User Input and Priority Selection
num_priorities = int(input("Enter the number of priorities: "))
priorities = []
for i in range(num_priorities):
    print(f"Select priority {i+1}:")
    print("1. School")
    print("2. Hospital")
    print("3. Railway Station")
    print("4. Metro Station")
    priority_input = int(input("Enter the number corresponding to the priority: "))
    if priority_input not in range(1, 5):
        print("Invalid input. Please enter a number between 1 and 4.")
        exit()
    priorities.append(priority_input)

# Preprocessing buildings data
X_buildings = buildings_data[['Latitude', 'Longitude']]

# Preprocessing neighborhood data
X_neighborhoods = neighborhood_data[['Latitude', 'Longitude']]

# KNN to find nearest neighbors
nearest_neighbors_indices = []
for i in range(num_priorities):
    knn = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(X_neighborhoods)
    _, indices = knn.kneighbors(X_buildings)
    nearest_neighbors_indices.append(indices.flatten())

# Function to calculate distance using Mapbox API
def get_distance(coords1, coords2):
    access_token = "YOUR_MAPBOX_ACCESS_TOKEN"  # Replace with your Mapbox access token
    url = f"https://api.mapbox.com/directions-matrix/v1/mapbox/driving/{coords1[1]},{coords1[0]};{coords2[1]},{coords2[0]}"
    params = {
        'access_token': access_token
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an HTTPError for bad status codes
        data = response.json()
        distance_meters = data['durations'][0][1]  # Distance in meters
        distance_km = distance_meters / 1000  # Convert meters to kilometers
        return distance_km
    except requests.exceptions.RequestException as e:
        print(f"Error occurred during API request: {e}")
        return None
    except (KeyError, IndexError) as e:
        print(f"Invalid data format in the API response: {e}")
        return None

# Filter buildings within 3 kilometers of any priority
suggested_buildings = buildings_data.copy()
for i in range(num_priorities):
    distances_km = []
    for j in range(len(buildings_data)):
        if j < len(nearest_neighbors_indices[i]):  # Check if the index is within the range of nearest_neighbors_indices[i]
            index = nearest_neighbors_indices[i][j]
            if index < len(neighborhood_data):
                building_coords = (suggested_buildings.loc[j, 'Latitude'], suggested_buildings.loc[j, 'Longitude'])
                neighborhood_coords = (neighborhood_data.iloc[index]['Latitude'], neighborhood_data.iloc[index]['Longitude'])
                distance = get_distance(building_coords, neighborhood_coords)
                if distance is not None:
                    distances_km.append(distance)
                else:
                    distances_km.append(float('inf'))  # Use a large number to represent infinite distance
            else:
                print(f"Invalid index: {index}")
        else:
            print(f"Index {j} out of range for nearest_neighbors_indices[{i}]")
    suggested_buildings[f'Nearest_Priority_{i+1}_Dist'] = distances_km

# Print top 5 recommended buildings within 3 kilometers
if len(suggested_buildings) > 0:
    print("Top 5 Recommended Buildings within 3 kilometers:")
    for index, row in suggested_buildings.head(5).iterrows():
        if index < len(suggested_buildings):
            print(f"Building ID: {row['prop_id']}, Name: {row['prop_name']}")
            for i in range(num_priorities):
                if row[f'Nearest_Priority_{i+1}_Dist'] is not None and row[f'Nearest_Priority_{i+1}_Dist'] <= 3:
                    if nearest_neighbors_indices[i][index] < len(neighborhood_data):
                        n_name = neighborhood_data.iloc[nearest_neighbors_indices[i][index]]['n_name']
                        print(f"Nearest {priority_mapping[priorities[i]]}: {n_name}, Distance: {row[f'Nearest_Priority_{i+1}_Dist']:.2f} km")
            print()
else:
    print("No results")

NOW I'LL EXPLAIN TO YOU THE LOGIC OF MY CODE THEN YOU CORRECT IT OR MODIFY IT ACCORDINGLY.
step1- First it will ask for number of priorities the user wants(eg: user types 2)
step2- Now it will ask you for the priorities and this will repeat according to the number user has chosen in step1. (eg: here in step-1 user chooses 2 ,  so the code will ask for the priority the user wants to choose 2 times. here suppose i chose school and hospital)
step3- Now the code will use knn and mapbox distance api to find the nearest school and hospital and the buildings with least distance between school and building and hospital and building and will recommend the top 5 out of the list.(remember either school or hospital won't be allowed. both should be at minimum distance from the building)
step4-  so after top 5 recommend buildings are shown in step3, we will show other suggested option. so after top 5 , we print ("other suggested options") and then print options where either of the chooen prioritiy has least distance from building.
step5- suppose if step3 fails, then print("top 5 recommended building") then print("No buildings, please change your neighborhood priority selection") and then repeat step4.

NOW USE THIS LOGIC AND CODE GIVEN ABOVE TO MAKE 1 PROPER CODE!!

another example:
WANT THAT SUPPOSE FOR EXAMPLE I CHHOSE NUMBER OF PRIOEIRITIES 3- RAILWAY HOSPITAL AND SCHOOL, NOW IT WILL SHOW ME BUILDINGS THAT  ARE AT MINIMUM DISTANCES FROM ALL THREE AND DISPLAY THEM AS TOP 5 RECOMMENDATIONS. AND TO DISPLAY OTHER SUGGESTED BUILDINGS, DISPLAY THE NEXT 5 AFTER THE TOP 5 RECOMMENDED

nOW USE ALL OF THIS LOGIC AND THE ABOVE CODE AND GIVE ME THE CODE THAT USES THIS LOGIC. AND REMEBER TO USE MAPBOX DISTANCE API AND SKLEARN KNN.


In [8]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import requests

# Load datasets
buildings_data = pd.read_csv("25_buildings.csv")
neighborhood_data = pd.read_csv("neighborhood.csv")

# Priority Mapping
priority_mapping = {
    1: "School",
    2: "Hospital",
    3: "Railway Station",
    4: "Metro Station"
}

# User Input and Priority Selection
num_priorities = int(input("Enter the number of priorities: "))
print("1: School,\n2: Hospital,\n3: Railway Station,\n4: Metro Station,\n Choose your priorities.")

priorities = [int(input(f"Select priority {i+1} (1-4): ")) for i in range(num_priorities)]

# Preprocessing buildings data
X_buildings = buildings_data[['Latitude', 'Longitude']]

# Preprocessing neighborhood data
X_neighborhoods = neighborhood_data[['Latitude', 'Longitude']]

# Function to calculate distance using Mapbox API
def get_distance(coords1, coords2):
    # Your Mapbox access token
    access_token = "pk.eyJ1Ijoic2FuaWthZCIsImEiOiJjbHVrNGdmcjgwZnB0MmpsNXUwdmcyMDR6In0.M4cFaJUCdMnCT0plVTnqVQ"  # Replace with your Mapbox access token
    url = "https://api.mapbox.com/directions/v5/mapbox/driving/"
    params = {
        'access_token': access_token,
        'coordinates': f"{coords1[1]},{coords1[0]};{coords2[1]},{coords2[0]}"
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an HTTPError for bad status codes
        data = response.json()
        distance_km = data['routes'][0]['distance'] / 1000  # Distance in kilometers
        return distance_km
    except requests.exceptions.RequestException as e:
        print(f"Error occurred during API request: {e}")
        return None

# KNN to find nearest neighbors
nearest_neighbors_indices = []
for priority in priorities:
    knn = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(X_neighborhoods)
    _, indices = knn.kneighbors(X_buildings)
    nearest_neighbors_indices.append(indices.flatten())

# Filter buildings within 3 kilometers of any priority
suggested_buildings = buildings_data.copy()
for i, priority in enumerate(priorities):
    distances_km = []
    for j in range(len(buildings_data)):
        if j < len(nearest_neighbors_indices[i]):
            index = nearest_neighbors_indices[i][j]
            if index < len(neighborhood_data):
                building_coords = (suggested_buildings.loc[j, 'Latitude'], suggested_buildings.loc[j, 'Longitude'])
                neighborhood_coords = (neighborhood_data.iloc[index]['Latitude'], neighborhood_data.iloc[index]['Longitude'])
                distance = get_distance(building_coords, neighborhood_coords)
                if distance is not None:
                    distances_km.append(distance)
                else:
                    distances_km.append(float('inf'))  # Use a large number to represent infinite distance
            else:
                print(f"Invalid index: {index}")
        else:
            print(f"Index {j} out of range for nearest_neighbors_indices[{i}]")
    suggested_buildings[f'Nearest_Priority_{priority}_Dist'] = distances_km

# Print top 5 recommended buildings within 3 kilometers
if len(suggested_buildings) > 0:
    print("Top 5 Recommended Buildings within 3 kilometers:")
    top_recommendations = suggested_buildings.nsmallest(5, f'Nearest_Priority_{priorities[0]}_Dist')
    for idx, row in top_recommendations.iterrows():
        print(f"Building: {row['prop_id']}, {row['prop_name']}")
        for priority in priorities:
            if row[f'Nearest_Priority_{priority}_Dist'] is not None and row[f'Nearest_Priority_{priority}_Dist'] <= 3:
                if nearest_neighbors_indices[priorities.index(priority)][idx] < len(neighborhood_data):
                    n_name = neighborhood_data.iloc[nearest_neighbors_indices[priorities.index(priority)][idx]]['n_name']
                    print(f"Nearest {priority_mapping[priority]}: {n_name}, Distance: {row[f'Nearest_Priority_{priority}_Dist']:.2f} km")
        print()
    # Other suggested options
    print("Other Suggested Buildings:")
    next_recommendations = suggested_buildings.nsmallest(10, f'Nearest_Priority_{priorities[0]}_Dist').iloc[5:]
    for idx, row in next_recommendations.iterrows():
        print(f"Building: {row['prop_id']}, {row['prop_name']}")
        for priority in priorities:
            if row[f'Nearest_Priority_{priority}_Dist'] is not None and row[f'Nearest_Priority_{priority}_Dist'] <= 3:
                if nearest_neighbors_indices[priorities.index(priority)][idx] < len(neighborhood_data):
                    n_name = neighborhood_data.iloc[nearest_neighbors_indices[priorities.index(priority)][idx]]['n_name']
                    print(f"Nearest {priority_mapping[priority]}: {n_name}, Distance: {row[f'Nearest_Priority_{priority}_Dist']:.2f} km")
        print()
else:
    print("No results")


1: School,
2: Hospital,
3: Railway Station,
4: Metro Station,
 Choose your priorities.
Error occurred during API request: 404 Client Error: Not Found for url: https://api.mapbox.com/directions/v5/mapbox/driving/?access_token=pk.eyJ1Ijoic2FuaWthZCIsImEiOiJjbHVrNGdmcjgwZnB0MmpsNXUwdmcyMDR6In0.M4cFaJUCdMnCT0plVTnqVQ&coordinates=72.9113553%2C19.0867632%3B72.9104529%2C19.0853873
Error occurred during API request: 404 Client Error: Not Found for url: https://api.mapbox.com/directions/v5/mapbox/driving/?access_token=pk.eyJ1Ijoic2FuaWthZCIsImEiOiJjbHVrNGdmcjgwZnB0MmpsNXUwdmcyMDR6In0.M4cFaJUCdMnCT0plVTnqVQ&coordinates=72.9095308%2C19.0877651%3B72.9096172%2C19.0874846
Error occurred during API request: 404 Client Error: Not Found for url: https://api.mapbox.com/directions/v5/mapbox/driving/?access_token=pk.eyJ1Ijoic2FuaWthZCIsImEiOiJjbHVrNGdmcjgwZnB0MmpsNXUwdmcyMDR6In0.M4cFaJUCdMnCT0plVTnqVQ&coordinates=72.915127%2C19.0869198%3B72.9140609%2C19.0858279
Error occurred during API request: 404 Clien

In [12]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import requests

# Load datasets
buildings_data = pd.read_csv("25_buildings.csv")
neighborhood_data = pd.read_csv("neighborhood.csv")

# Priority Mapping
priority_mapping = {
    1: "School",
    2: "Hospital",
    3: "Railway Station",
    4: "Metro Station"
}

# User Input and Priority Selection
num_priorities = int(input("Enter the number of priorities (1-4): "))
priorities = [int(input(f"Select priority {i+1} (1-4): ")) for i in range(num_priorities)]

# Preprocessing buildings data
X_buildings = buildings_data[['Latitude', 'Longitude']]

# Preprocessing neighborhood data
X_neighborhoods = neighborhood_data[['Latitude', 'Longitude']]

# Function to calculate distance using Mapbox API
def get_distance(coords1, coords2):
    access_token = "pk.eyJ1Ijoic2FuaWthZCIsImEiOiJjbHVrNGdmcjgwZnB0MmpsNXUwdmcyMDR6In0.M4cFaJUCdMnCT0plVTnqVQ"  # Replace with your Mapbox access token
    url = f"https://api.mapbox.com/directions-matrix/v1/mapbox/driving/{coords1[1]},{coords1[0]};{coords2[1]},{coords2[0]}"
    params = {
        'access_token': access_token
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an HTTPError for bad status codes
        data = response.json()
        distance_meters = data['durations'][0][1]  # Distance in meters
        distance_km = distance_meters / 1000  # Convert meters to kilometers
        return distance_km
    except requests.exceptions.RequestException as e:
        print(f"Error occurred during API request: {e}")
        return None
    except (KeyError, IndexError) as e:
        print(f"Invalid data format in the API response: {e}")
        return None

# KNN to find nearest neighbors
nearest_neighbors_indices = []
for priority in priorities:
    knn = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(X_neighborhoods)
    _, indices = knn.kneighbors(X_buildings)
    nearest_neighbors_indices.append(indices.flatten())

# Filter buildings within 3 kilometers of all priorities
suggested_buildings = buildings_data.copy()
for i, priority in enumerate(priorities):
    distances_km = []
    for j in range(len(buildings_data)):
        if j < len(nearest_neighbors_indices[i]):
            index = nearest_neighbors_indices[i][j]
            if index < len(neighborhood_data):
                building_coords = (suggested_buildings.loc[j, 'Latitude'], suggested_buildings.loc[j, 'Longitude'])
                neighborhood_coords = (neighborhood_data.iloc[index]['Latitude'], neighborhood_data.iloc[index]['Longitude'])
                distance = get_distance(building_coords, neighborhood_coords)
                if distance is not None:
                    distances_km.append(distance)
                else:
                    distances_km.append(float('inf'))  # Use a large number to represent infinite distance
            else:
                print(f"Invalid index: {index}")
        else:
            print(f"Index {j} out of range for nearest_neighbors_indices[{i}]")
    suggested_buildings[f'Nearest_Priority_{priority}_Dist'] = distances_km

# Print top 5 recommended buildings within 3 kilometers for all priorities
if len(suggested_buildings) > 0:
    top_recommendations = suggested_buildings
    for priority in priorities:
        top_recommendations = top_recommendations.loc[top_recommendations[f'Nearest_Priority_{priority}_Dist'] <= 3]

    top_recommendations = top_recommendations.head(5)
    if len(top_recommendations) > 0:
        print("Top 5 Recommended Buildings within 3 kilometers:")
        for idx, row in top_recommendations.iterrows():
            print(f"Building: {row['prop_id']}, {row['prop_name']}")
            for priority in priorities:
                n_index = nearest_neighbors_indices[priorities.index(priority)][idx]
                n_id = neighborhood_data.iloc[n_index]['n_id']  # Retrieve n_id from neighborhood_data
                n_name = neighborhood_data.iloc[n_index]['n_name']
                distance = row[f'Nearest_Priority_{priority}_Dist']
                print(f"Nearest {priority_mapping[priority]}: ({n_id}), {n_name}, Distance: {distance:.2f} km")
            print()
    else:
        print("No results within 3 kilometers for all selected priorities.")
else:
    print("No results")


Top 5 Recommended Buildings within 3 kilometers:
Building: 1, Kesar Scion
Nearest Railway Station: (48), Khus-Hal Hospital
, Distance: 0.05 km
Nearest Hospital: (48), Khus-Hal Hospital
, Distance: 0.05 km
Nearest School: (48), Khus-Hal Hospital
, Distance: 0.05 km

Building: 2, Alag Olive

Nearest Railway Station: (3), SHRI INDRADEO SINGH INTERNATIONAL SCHOOL, Distance: 0.00 km
Nearest Hospital: (3), SHRI INDRADEO SINGH INTERNATIONAL SCHOOL, Distance: 0.00 km
Nearest School: (3), SHRI INDRADEO SINGH INTERNATIONAL SCHOOL, Distance: 0.00 km

Building: 3, Ugam Gurukrupa
, Distance: 0.20 kmtion: (19), Suyash Vidyalaya
, Distance: 0.20 km19), Suyash Vidyalaya
, Distance: 0.20 km), Suyash Vidyalaya

Building: 4, Mohite Marvel
Nearest Railway Station: (17), Dominic Savio Vidyalaya, Distance: 0.13 km
Nearest Hospital: (17), Dominic Savio Vidyalaya, Distance: 0.13 km
Nearest School: (17), Dominic Savio Vidyalaya, Distance: 0.13 km

Building: 5, Eastsyde
Nearest Railway Station: (17), Dominic Sa

In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import requests

# Load datasets
buildings_data = pd.read_csv("25_buildings.csv")
neighborhood_data = pd.read_csv("neighborhood.csv")

# User Input and Priority Selection
num_priorities = int(input("Enter the number of priorities: "))
selected_priorities = []
priority_mapping = {
    1: "School",
    2: "Hospital",
    3: "Railway Station",
    4: "Metro Station"
}

for i in range(num_priorities):
    print(f"Select priority {i+1}:")
    print("1. School")
    print("2. Hospital")
    print("3. Railway Station")
    print("4. Metro Station")
    priority_input = int(input("Enter the number corresponding to the priority: "))
    if priority_input not in range(1, 5):
        print("Invalid input. Please enter a number between 1 and 4.")
        exit()
    selected_priorities.append(priority_input)

# Preprocessing buildings data
X_buildings = buildings_data[['Latitude', 'Longitude']]

# Preprocessing neighborhood data
X_neighborhoods = neighborhood_data[['Latitude', 'Longitude']]

# KNN to find nearest neighbors
nearest_neighbors_indices = []
for priority in selected_priorities:
    knn = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(X_neighborhoods)
    _, indices = knn.kneighbors(X_buildings)
    nearest_neighbors_indices.append(indices.flatten())

# Function to calculate distance using Mapbox API
def get_distance(coords1, coords2):
    # Replace this section with the correct implementation using Mapbox API
    access_token = "pk.eyJ1Ijoic2FuaWthZCIsImEiOiJjbHVrNGdmcjgwZnB0MmpsNXUwdmcyMDR6In0.M4cFaJUCdMnCT0plVTnqVQ"  # Replace with your Mapbox access token
    url = f"https://api.mapbox.com/directions-matrix/v1/mapbox/driving/{coords1[1]},{coords1[0]};{coords2[1]},{coords2[0]}"
    params = {
        'access_token': access_token
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an HTTPError for bad status codes
        data = response.json()
        distance_meters = data['durations'][0][1]  # Distance in meters
        distance_km = distance_meters / 1000  # Convert meters to kilometers
        return distance_km
    except requests.exceptions.RequestException as e:
        print(f"Error occurred during API request: {e}")
        return None
    except (KeyError, IndexError) as e:
        print(f"Invalid data format in the API response: {e}")
        return None

# Filter buildings within 3 kilometers of any priority
suggested_buildings = buildings_data.copy()
for i in range(num_priorities):
    distances_km = []
    for j in range(len(buildings_data)):
        if j < len(nearest_neighbors_indices[i]):  # Check if the index is within the range of nearest_neighbors_indices[i]
            index = nearest_neighbors_indices[i][j]
            if index < len(neighborhood_data):
                building_coords = (suggested_buildings.loc[j, 'Latitude'], suggested_buildings.loc[j, 'Longitude'])
                neighborhood_coords = (neighborhood_data.iloc[index]['Latitude'], neighborhood_data.iloc[index]['Longitude'])
                distance = get_distance(building_coords, neighborhood_coords)
                if distance is not None:
                    distances_km.append(distance)
                else:
                    distances_km.append(float('inf'))  # Use a large number to represent infinite distance
            else:
                print(f"Invalid index: {index}")
        else:
            print(f"Index {j} out of range for nearest_neighbors_indices[{i}]")
    suggested_buildings[f'Nearest_Priority_{i+1}_Dist'] = distances_km

# Print top 5 recommended buildings within 3 kilometers
if len(suggested_buildings) > 0:
    print("Top 5 Recommended Buildings within 3 kilometers:")
    for index, row in suggested_buildings.head(5).iterrows():
        if index < len(suggested_buildings):
            print(f"Building ID: {row['prop_id']}, Name: {row['prop_name']}")
            for i in range(num_priorities):
                if row[f'Nearest_Priority_{i+1}_Dist'] is not None and row[f'Nearest_Priority_{i+1}_Dist'] <= 3:
                    if nearest_neighbors_indices[i][index] < len(neighborhood_data):
                        n_name = neighborhood_data.iloc[nearest_neighbors_indices[i][index]]['n_name']
                        print(f"Nearest {priority_mapping[selected_priorities[i]]}: {n_name}, Distance: {row[f'Nearest_Priority_{i+1}_Dist']:.2f} km")
            print()
else:
    print("No results")


Select priority 1:
1. School
2. Hospital
3. Railway Station
4. Metro Station
Select priority 2:
1. School
2. Hospital
3. Railway Station
4. Metro Station
Select priority 3:
1. School
2. Hospital
3. Railway Station
4. Metro Station
Top 5 Recommended Buildings within 3 kilometers:
Building ID: 1, Name: Kesar Scion
Nearest Railway Station: Khus-Hal Hospital
, Distance: 0.05 km
Nearest Hospital: Khus-Hal Hospital
, Distance: 0.05 km
Nearest School: Khus-Hal Hospital
, Distance: 0.05 km

Building ID: 2, Name: Alag Olive

Nearest Railway Station: SHRI INDRADEO SINGH INTERNATIONAL SCHOOL, Distance: 0.00 km
Nearest Hospital: SHRI INDRADEO SINGH INTERNATIONAL SCHOOL, Distance: 0.00 km
Nearest School: SHRI INDRADEO SINGH INTERNATIONAL SCHOOL, Distance: 0.00 km

Building ID: 3, Name: Ugam Gurukrupa
, Distance: 0.20 kmtion: Suyash Vidyalaya
, Distance: 0.20 kmuyash Vidyalaya
, Distance: 0.20 kmash Vidyalaya

Building ID: 4, Name: Mohite Marvel
Nearest Railway Station: Dominic Savio Vidyalaya, Dist

In [3]:
import pandas as pd
import requests
import json

# Load datasets
buildings_data = pd.read_csv("buildings_dataset.csv")
neighborhood_data = pd.read_csv("neighborhood.csv")

# User Input and Priority Selection
num_priorities = 3  # Assuming 3 priorities based on the provided dataset
selected_priorities = ["School", "Hospital", "Railway Station"]  # Adjust based on the available n_type values

# Preprocessing buildings data
X_buildings = buildings_data[['Latitude', 'Longitude']]

# Preprocessing neighborhood data
neighborhood_priorities = ["School", "Hospital", "Railway Station"]  # Assuming these are the priorities in the neighborhood dataset
X_neighborhoods = neighborhood_data[neighborhood_data['n_type'].isin(neighborhood_priorities)][['Latitude', 'Longitude']]

# Obtain API key from Mapbox account
mapbox_api_key = 'pk.eyJ1Ijoic2FuaWthZCIsImEiOiJjbHVrNGdmcjgwZnB0MmpsNXUwdmcyMDR6In0.M4cFaJUCdMnCT0plVTnqVQ'

# Function to calculate driving distance using Mapbox Matrix API
def get_driving_distance(coords1, coords2):
    base_url = 'https://api.mapbox.com/directions-matrix/v1/mapbox/driving/'
    coordinates = f'{coords1[1]},{coords1[0]};{coords2[1]},{coords2[0]}'
    access_token = f'access_token={mapbox_api_key}'
    request_url = f'{base_url}{coordinates}?{access_token}'
    response = requests.get(request_url)
    data = json.loads(response.text)
    driving_distance_km = data['durations'][0][1] / 60 / 60  # Convert seconds to hours
    return driving_distance_km

# Filter buildings within 3 kilometers of any priority
suggested_buildings = buildings_data.copy()
for i in range(num_priorities):
    distances_km = []
    for j in range(len(buildings_data)):
        if j < len(X_neighborhoods):
            building_coords = (suggested_buildings.loc[j, 'Latitude'], suggested_buildings.loc[j, 'Longitude'])
            neighborhood_coords = (X_neighborhoods.iloc[j]['Latitude'], X_neighborhoods.iloc[j]['Longitude'])
            distance = get_driving_distance(building_coords, neighborhood_coords)
            distances_km.append(distance)
    suggested_buildings[f'Nearest_Priority_{i+1}_Dist'] = distances_km

# Print top 5 recommended buildings within 3 kilometers
if len(suggested_buildings) > 0:
    print("Top 5 Recommended Buildings within 3 kilometers:")
    for index, row in suggested_buildings.head(5).iterrows():
        if index < len(suggested_buildings):
            print(f"Building ID: {row['prop_id']}, Name: {row['prop_name']}")
            for i in range(num_priorities):
                if row[f'Nearest_Priority_{i+1}_Dist'] is not None and row[f'Nearest_Priority_{i+1}_Dist'] <= 3:
                    n_name = neighborhood_data.iloc[i]['n_name']
                    print(f"Nearest {selected_priorities[i]}: {n_name}, Distance: {row[f'Nearest_Priority_{i+1}_Dist']:.2f} km")
            print()
else:
    print("No results")


KeyError: 'durations'

this is my dataset:
Dataset1- buildings_dataset.csv: 
prop_id	prop_name	prop_addr	Latitude	Longitude	 price	 square_footage	 bhk	 rent	 sale_type	 furnishing_type	 reserved_parking	 kids_play_area	 gym	 swimming_pool	 club_house --
1	Ashapura apartment	B/6 ASHAPURA APT. NEW, Andheri - Ghatkopar Rd, Maneklal Estate, W, Mumbai, Maharashtra 400086	19.0904851	72.9012085	10691758	713	1	24728	 resale	 unfurnished	TRUE	TRUE	TRUE	FALSE	TRUE

Dataset2- neighborhood.csv:
 n_id	n_name	n_addr	Latitude	Longitude	n_type--
1	Garodia International Centre for Learning Mumbai	"Garodia International Centre for Learning Mumbai, 55, 90 Feet Rd, Garodia Nagar, Ghatkopar East, Mumbai, Maharashtra 400077
"	19.0748994	72.9060813	School
it has 94 more entries that include school hospital metro railway(n_type attribute is where these 4 are mentioned accordingly)

now i hope you got clarity as i didn't mention how you will choose the priority type(here n_type) now please solve the error in code



In [4]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import requests

# Step 2: Load Datasets
buildings_data = pd.read_csv("25_buildings.csv")
neighborhood_data = pd.read_csv("neighborhood.csv")

# Step 3: User Input and Priority Selection
num_priorities = int(input("Enter the number of priorities: "))
priorities = []
priority_mapping = {1: "School", 2: "Hospital", 3: "Railway Station", 4: "Metro Station"}
for i in range(num_priorities):
    print(f"Select priority {i+1}:")
    print("1. School")
    print("2. Hospital")
    print("3. Railway Station")
    print("4. Metro Station")
    priority_input = int(input("Enter the number corresponding to the priority: "))
    if priority_input not in range(1, 5):
        print("Invalid input. Please enter a number between 1 and 4.")
        exit()
    priorities.append(priority_input)

# Step 4: KNN to Find Nearest Neighbors
X_buildings = buildings_data[['Latitude', 'Longitude']]
X_neighborhoods = neighborhood_data[['Latitude', 'Longitude']]
nearest_neighbors_indices = []
for i in range(num_priorities):
    knn = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(X_neighborhoods)
    _, indices = knn.kneighbors(X_buildings)
    nearest_neighbors_indices.append(indices.flatten())

# Step 5: Function to Calculate Distance using Mapbox API
def get_distance(coords1, coords2):
    access_token = "pk.eyJ1Ijoic2FuaWthZCIsImEiOiJjbHVrNGdmcjgwZnB0MmpsNXUwdmcyMDR6In0.M4cFaJUCdMnCT0plVTnqVQ"  # Replace with your Mapbox access token
    url = f"https://api.mapbox.com/directions-matrix/v1/mapbox/driving/{coords1[1]},{coords1[0]};{coords2[1]},{coords2[0]}"
    params = {
        'access_token': access_token
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an HTTPError for bad status codes
        data = response.json()
        distance_meters = data['durations'][0][1]  # Distance in meters
        distance_km = distance_meters / 1000  # Convert meters to kilometers
        return distance_km
    except requests.exceptions.RequestException as e:
        print(f"Error occurred during API request: {e}")
        return None
    except (KeyError, IndexError) as e:
        print(f"Invalid data format in the API response: {e}")
        return None

# Step 6: Filter Buildings within 3 Kilometers of Any Priority
suggested_buildings = buildings_data.copy()
for i in range(num_priorities):
    distances_km = []
    for j in range(len(buildings_data)):
        if j < len(nearest_neighbors_indices[i]):  # Check if the index is within the range of nearest_neighbors_indices[i]
            index = nearest_neighbors_indices[i][j]
            if index < len(neighborhood_data):
                building_coords = (suggested_buildings.loc[j, 'Latitude'], suggested_buildings.loc[j, 'Longitude'])
                neighborhood_coords = (neighborhood_data.iloc[index]['Latitude'], neighborhood_data.iloc[index]['Longitude'])
                distance = get_distance(building_coords, neighborhood_coords)
                if distance is not None:
                    distances_km.append(distance)
                else:
                    distances_km.append(float('inf'))  # Use a large number to represent infinite distance
            else:
                print(f"Invalid index: {index}")
        else:
            print(f"Index {j} out of range for nearest_neighbors_indices[{i}]")
    suggested_buildings[f'Nearest_Priority_{i+1}_Dist'] = distances_km

# Step 7: Print Top 5 Recommended Buildings and Other Suggested Options
# Print top 5 recommended buildings within 3 kilometers
if len(suggested_buildings) > 0:
    print("Top 5 Recommended Buildings within 3 kilometers:")
    for index, row in suggested_buildings.head(5).iterrows():
        if index < len(suggested_buildings):
            print(f"Building ID: {row['prop_id']}, Name: {row['prop_name']}")
            for i in range(num_priorities):
                if row[f'Nearest_Priority_{i+1}_Dist'] is not None and row[f'Nearest_Priority_{i+1}_Dist'] <= 3:
                    if nearest_neighbors_indices[i][index] < len(neighborhood_data):
                        n_name = neighborhood_data.iloc[nearest_neighbors_indices[i][index]]['n_name']
                        print(f"Nearest {priority_mapping[priorities[i]]}: {n_name}, Distance: {row[f'Nearest_Priority_{i+1}_Dist']:.2f} km")
            print()
else:
    print("No results")

# Modify the code to display other suggested options
if len(suggested_buildings) > 5:
    print("Other Suggested Options:")
    for index, row in suggested_buildings.iloc[5:10].iterrows():
        if index < len(suggested_buildings):
            print(f"Building ID: {row['prop_id']}, Name: {row['prop_name']}")
            # Display other suggested options based on priorities
    print()
else:
    print("No additional suggested options")


Select priority 1:
1. School
2. Hospital
3. Railway Station
4. Metro Station
Select priority 2:
1. School
2. Hospital
3. Railway Station
4. Metro Station
Top 5 Recommended Buildings within 3 kilometers:
Building ID: 1, Name: Kesar Scion
Nearest Hospital: Khus-Hal Hospital
, Distance: 0.05 km
Nearest School: Khus-Hal Hospital
, Distance: 0.05 km

Building ID: 2, Name: Alag Olive

Nearest Hospital: SHRI INDRADEO SINGH INTERNATIONAL SCHOOL, Distance: 0.00 km
Nearest School: SHRI INDRADEO SINGH INTERNATIONAL SCHOOL, Distance: 0.00 km

Building ID: 3, Name: Ugam Gurukrupa
, Distance: 0.20 kmuyash Vidyalaya
, Distance: 0.20 kmash Vidyalaya

Building ID: 4, Name: Mohite Marvel
Nearest Hospital: Dominic Savio Vidyalaya, Distance: 0.13 km
Nearest School: Dominic Savio Vidyalaya, Distance: 0.13 km

Building ID: 5, Name: Eastsyde
Nearest Hospital: Dominic Savio Vidyalaya, Distance: 0.23 km
Nearest School: Dominic Savio Vidyalaya, Distance: 0.23 km

Other Suggested Options:
Building ID: 6, Name: S

In [2]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import requests

def get_distance(coords1, coords2):
    access_token = "pk.eyJ1Ijoic2FuaWthZCIsImEiOiJjbHVrNGdmcjgwZnB0MmpsNXUwdmcyMDR6In0.M4cFaJUCdMnCT0plVTnqVQ"  # Replace with your Mapbox access token
    url = f"https://api.mapbox.com/directions-matrix/v1/mapbox/driving/{coords1[1]},{coords1[0]};{coords2[1]},{coords2[0]}"
    params = {
        'access_token': access_token
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        distance_meters = data['durations'][0][1]
        distance_km = distance_meters / 1000
        return distance_km
    except requests.exceptions.RequestException as e:
        print(f"Error occurred during API request: {e}")
        return None
    except (KeyError, IndexError) as e:
        print(f"Invalid data format in the API response: {e}")
        return None

buildings_data = pd.read_csv("25_buildings.csv")
neighborhood_data = pd.read_csv("neighborhood.csv")

print("Select your 2 preferred neighborhoods:")
print("1. School")
print("2. Hospital")
print("3. Railway Station")
print("4. Metro Station")

priority_mapping = {
    1: "School",
    2: "Hospital",
    3: "Railway Station",
    4: "Metro Station"
}

priority_input = input("Enter the numbers corresponding to your two preferred neighborhoods separated by a comma: ")
priority_input = [int(x) for x in priority_input.split(',') if int(x) in priority_mapping]

if len(priority_input) != 2:
    print("Please enter exactly 2 valid numbers between 1 and 4 corresponding to the preferred neighborhoods.")
else:
    selected_priorities = [priority_mapping[priority] for priority in priority_input]
    filtered_neighborhood_data = neighborhood_data[neighborhood_data['n_type'].isin(selected_priorities)].reset_index(drop=True)

    X_buildings = buildings_data[['Latitude', 'Longitude']]
    X_neighborhood = filtered_neighborhood_data[['Latitude', 'Longitude']]

    knn = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(X_neighborhood)
    _, indices = knn.kneighbors(X_buildings)

    distances_km = []
    for i in range(len(buildings_data)):
        coords1 = (buildings_data.loc[i, 'Latitude'], buildings_data.loc[i, 'Longitude'])
        coords2 = (filtered_neighborhood_data.loc[indices[i][0], 'Latitude'], filtered_neighborhood_data.loc[indices[i][0], 'Longitude'])
        distance_km = get_distance(coords1, coords2)
        distances_km.append(distance_km)

    buildings_data['Nearest_Neighborhood_ID'] = filtered_neighborhood_data.iloc[indices.flatten()]['n_id'].values
    buildings_data['Distance_to_Nearest_Neighborhood'] = distances_km

    sorted_buildings_data = buildings_data.sort_values(by='Distance_to_Nearest_Neighborhood')

if len(sorted_buildings_data) > 0:
    print("\nTop Recommended Building:")
    top_building = sorted_buildings_data.head(1)
    nearest_priority_1 = filtered_neighborhood_data[filtered_neighborhood_data['n_id'] == top_building['Nearest_Neighborhood_ID'].values[0]]['n_name'].iloc[0]
    nearest_priority_2 = filtered_neighborhood_data[filtered_neighborhood_data['n_type'] == selected_priorities[1]]['n_name'].iloc[0]
    prop_id = top_building['Building ID'].values[0]  # Assuming 'Building ID' is the column name for building ID
    prop_name = top_building['Building Name'].values[0]

    print(f"Building ID: {prop_id}")
    print(f"Building Name: {prop_name}")
    print(f"Latitude: {top_building['Latitude'].values[0]}")
    print(f"Longitude: {top_building['Longitude'].values[0]}")
    print(f"Nearest Priority 1 ({selected_priorities[0]}): {nearest_priority_1}")
    print(f"Nearest Priority 2 ({selected_priorities[1]}): {nearest_priority_2}")
    print(f"Distance to Nearest Neighborhood: {top_building['Distance_to_Nearest_Neighborhood'].values[0]} km")
else:
    print("No results found.")

Select your 2 preferred neighborhoods:
1. School
2. Hospital
3. Railway Station
4. Metro Station

Top Recommended Building:


KeyError: 'Building ID'

In [3]:
#2 input try
#FINAL ONE FOR MULTIPLE PRIORITIES
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import requests

# Define the get_distance function
def get_distance(coords1, coords2):
    access_token = "pk.eyJ1Ijoic2FuaWthZCIsImEiOiJjbHVrNGdmcjgwZnB0MmpsNXUwdmcyMDR6In0.M4cFaJUCdMnCT0plVTnqVQ"  # Replace with your Mapbox access token
    url = f"https://api.mapbox.com/directions-matrix/v1/mapbox/driving/{coords1[1]},{coords1[0]};{coords2[1]},{coords2[0]}"
    params = {
        'access_token': access_token
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an HTTPError for bad status codes
        data = response.json()
        distance_meters = data['durations'][0][1]  # Distance in meters
        distance_km = distance_meters / 1000  # Convert meters to kilometers
        return distance_km
    except requests.exceptions.RequestException as e:
        print(f"Error occurred during API request: {e}")
        return None
    except (KeyError, IndexError) as e:
        print(f"Invalid data format in the API response: {e}")
        return None

# Load datasets
buildings_data = pd.read_csv("25_buildings.csv")
neighborhood_data = pd.read_csv("neighborhood.csv")

# Select the preferred neighborhood priorities
print("Select your two preferred neighborhood priorities:")
print("1. School")
print("2. Hospital")
print("3. Railway Station")
print("4. Metro Station")

priority_mapping = {
    1: "School",
    2: "Hospital",
    3: "Railway Station",
    4: "Metro Station"
}

priority_input1 = int(input("Enter the number corresponding to your first preferred priority: "))
priority_input2 = int(input("Enter the number corresponding to your second preferred priority: "))

# Validate inputs
if priority_input1 not in priority_mapping or priority_input2 not in priority_mapping or priority_input1 == priority_input2:
    print("Invalid inputs. Please enter two different numbers between 1 and 4.")
else:
    selected_priority1 = priority_mapping[priority_input1]
    selected_priority2 = priority_mapping[priority_input2]
    print("Your selected priorities are:", selected_priority1, "and", selected_priority2)

    # Filtering neighborhood.csv based on selected priorities
    filtered_neighborhood_data1 = neighborhood_data[neighborhood_data['n_type'] == selected_priority1].reset_index(drop=True)
    filtered_neighborhood_data2 = neighborhood_data[neighborhood_data['n_type'] == selected_priority2].reset_index(drop=True)

    # Preprocessing buildings_dataset data
    X_buildings = buildings_data[['Latitude', 'Longitude']]

    # Preprocessing neighborhood.csv for both priorities
    X_neighborhood1 = filtered_neighborhood_data1[['Latitude', 'Longitude']]
    X_neighborhood2 = filtered_neighborhood_data2[['Latitude', 'Longitude']]

    # KNN to find nearest neighbors for both priorities
    knn1 = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(X_neighborhood1)
    _, indices1 = knn1.kneighbors(X_buildings)

    knn2 = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(X_neighborhood2)
    _, indices2 = knn2.kneighbors(X_buildings)

    # Calculate distances using Mapbox API for both priorities
    distances_km = []
    for i in range(len(buildings_data)):
        coords1 = (buildings_data.loc[i, 'Latitude'], buildings_data.loc[i, 'Longitude'])
        coords2_1 = (filtered_neighborhood_data1.loc[indices1[i][0], 'Latitude'], filtered_neighborhood_data1.loc[indices1[i][0], 'Longitude'])
        coords2_2 = (filtered_neighborhood_data2.loc[indices2[i][0], 'Latitude'], filtered_neighborhood_data2.loc[indices2[i][0], 'Longitude'])
        
        distance_km1 = get_distance(coords1, coords2_1)
        distance_km2 = get_distance(coords1, coords2_2)
        
        combined_distance = distance_km1 + distance_km2
        distances_km.append(combined_distance)

    # Add nearest neighborhood information and distance to buildings dataset
    buildings_data['Nearest_Neighborhood_ID1'] = filtered_neighborhood_data1.iloc[indices1.flatten()]['n_id'].values
    buildings_data['Nearest_Neighborhood_ID2'] = filtered_neighborhood_data2.iloc[indices2.flatten()]['n_id'].values
    buildings_data['Distance_to_Nearest_Neighborhood_Combined'] = distances_km

    # Sort buildings by combined distance
    sorted_buildings_data = buildings_data.sort_values(by='Distance_to_Nearest_Neighborhood_Combined')

    # Print top 5 recommended buildings
    if len(sorted_buildings_data) > 0:
        print("\nTop 5 Recommended Buildings:")
        for index, row in sorted_buildings_data.head(5).iterrows():
            nearest_neighborhood_name1 = filtered_neighborhood_data1[
                filtered_neighborhood_data1['n_id'] == row['Nearest_Neighborhood_ID1']]['n_name'].iloc[0]
            nearest_neighborhood_name2 = filtered_neighborhood_data2[
                filtered_neighborhood_data2['n_id'] == row['Nearest_Neighborhood_ID2']]['n_name'].iloc[0]
            print(f"Building ID: {row['prop_id']}, Name: {row['prop_name']}")
            print(f"Nearby {selected_priority1}: {row['Nearest_Neighborhood_ID1']}, {nearest_neighborhood_name1}")
            print(f"Nearby {selected_priority2}: {row['Nearest_Neighborhood_ID2']}, {nearest_neighborhood_name2}")
            # print(f"Combined Distance: {row['Distance_to_Nearest_Neighborhood_Combined']:.2f} km")
            print()
    else:
        print("No results")

Select your two preferred neighborhood priorities:
1. School
2. Hospital
3. Railway Station
4. Metro Station
Your selected priorities are: School and Metro Station

Top 5 Recommended Buildings:
Building ID: 2, Name: Alag Olive

Nearby School: 3, SHRI INDRADEO SINGH INTERNATIONAL SCHOOL
Nearby Metro Station: 85, Ghatkopar Metro Station

Building ID: 17, Name: Alag Olive

Nearby School: 3, SHRI INDRADEO SINGH INTERNATIONAL SCHOOL
Nearby Metro Station: 85, Ghatkopar Metro Station

Building ID: 19, Name: SKYLINE MAGNUS
Nearby School: 10, D J Doshi Gurukul English Medium High School and Junior College of Commerce
Nearby Metro Station: 86, Ghatkopar Metro Station (East side)


Building ID: 14, Name: SKYLINE MAGNUS
Nearby School: 10, D J Doshi Gurukul English Medium High School and Junior College of Commerce
Nearby Metro Station: 86, Ghatkopar Metro Station (East side)


Building ID: 22, Name: Rajshree 11 East

Nearby School: 22, Sheth Virchand Dhanji Secondary English Medium School

Nearby M

In [5]:
#all buildings list
#2 input try
#FINAL ONE FOR MULTIPLE PRIORITIES
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import requests

# Define the get_distance function
def get_distance(coords1, coords2):
    access_token = "pk.eyJ1Ijoic2FuaWthZCIsImEiOiJjbHVrNGdmcjgwZnB0MmpsNXUwdmcyMDR6In0.M4cFaJUCdMnCT0plVTnqVQ"  # Replace with your Mapbox access token
    url = f"https://api.mapbox.com/directions-matrix/v1/mapbox/driving/{coords1[1]},{coords1[0]};{coords2[1]},{coords2[0]}"
    params = {
        'access_token': access_token
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an HTTPError for bad status codes
        data = response.json()
        distance_meters = data['durations'][0][1]  # Distance in meters
        distance_km = distance_meters / 1000  # Convert meters to kilometers
        return distance_km
    except requests.exceptions.RequestException as e:
        print(f"Error occurred during API request: {e}")
        return None
    except (KeyError, IndexError) as e:
        print(f"Invalid data format in the API response: {e}")
        return None

# Load datasets
buildings_data = pd.read_csv("25_buildings.csv")
neighborhood_data = pd.read_csv("neighborhood.csv")

# Select the preferred neighborhood priorities
print("Select your two preferred neighborhood priorities:")
print("1. School")
print("2. Hospital")
print("3. Railway Station")
print("4. Metro Station")

priority_mapping = {
    1: "School",
    2: "Hospital",
    3: "Railway Station",
    4: "Metro Station"
}

priority_input1 = int(input("Enter the number corresponding to your first preferred priority: "))
priority_input2 = int(input("Enter the number corresponding to your second preferred priority: "))

# Validate inputs
if priority_input1 not in priority_mapping or priority_input2 not in priority_mapping or priority_input1 == priority_input2:
    print("Invalid inputs. Please enter two different numbers between 1 and 4.")
else:
    selected_priority1 = priority_mapping[priority_input1]
    selected_priority2 = priority_mapping[priority_input2]
    print("Your selected priorities are:", selected_priority1, "and", selected_priority2)

    # Filtering neighborhood.csv based on selected priorities
    filtered_neighborhood_data1 = neighborhood_data[neighborhood_data['n_type'] == selected_priority1].reset_index(drop=True)
    filtered_neighborhood_data2 = neighborhood_data[neighborhood_data['n_type'] == selected_priority2].reset_index(drop=True)

    # Preprocessing buildings_dataset data
    X_buildings = buildings_data[['Latitude', 'Longitude']]

    # Preprocessing neighborhood.csv for both priorities
    X_neighborhood1 = filtered_neighborhood_data1[['Latitude', 'Longitude']]
    X_neighborhood2 = filtered_neighborhood_data2[['Latitude', 'Longitude']]

    # KNN to find nearest neighbors for both priorities
    knn1 = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(X_neighborhood1)
    _, indices1 = knn1.kneighbors(X_buildings)

    knn2 = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(X_neighborhood2)
    _, indices2 = knn2.kneighbors(X_buildings)

    # Calculate distances using Mapbox API for both priorities
    distances_km = []
    for i in range(len(buildings_data)):
        coords1 = (buildings_data.loc[i, 'Latitude'], buildings_data.loc[i, 'Longitude'])
        coords2_1 = (filtered_neighborhood_data1.loc[indices1[i][0], 'Latitude'], filtered_neighborhood_data1.loc[indices1[i][0], 'Longitude'])
        coords2_2 = (filtered_neighborhood_data2.loc[indices2[i][0], 'Latitude'], filtered_neighborhood_data2.loc[indices2[i][0], 'Longitude'])
        
        distance_km1 = get_distance(coords1, coords2_1)
        distance_km2 = get_distance(coords1, coords2_2)
        
        combined_distance = distance_km1 + distance_km2
        distances_km.append(combined_distance)

    # Add nearest neighborhood information and distance to buildings dataset
    buildings_data['Nearest_Neighborhood_ID1'] = filtered_neighborhood_data1.iloc[indices1.flatten()]['n_id'].values
    buildings_data['Nearest_Neighborhood_ID2'] = filtered_neighborhood_data2.iloc[indices2.flatten()]['n_id'].values
    buildings_data['Distance_to_Nearest_Neighborhood_Combined'] = distances_km

    # Sort buildings by combined distance
    sorted_buildings_data = buildings_data.sort_values(by='Distance_to_Nearest_Neighborhood_Combined')

    # Print top 5 recommended buildings
    if len(sorted_buildings_data) > 0:
        print("\nTop 5 Recommended Buildings:")
    for index, row in sorted_buildings_data.iterrows():  # Remove .head(5) to print all buildings
            nearest_neighborhood_name1 = filtered_neighborhood_data1[
                filtered_neighborhood_data1['n_id'] == row['Nearest_Neighborhood_ID1']]['n_name'].iloc[0]
            nearest_neighborhood_name2 = filtered_neighborhood_data2[
                filtered_neighborhood_data2['n_id'] == row['Nearest_Neighborhood_ID2']]['n_name'].iloc[0]
            print(f"Building ID: {row['prop_id']}, Name: {row['prop_name']}")
            print(f"Nearby {selected_priority1}: {row['Nearest_Neighborhood_ID1']}, {nearest_neighborhood_name1}")
            print(f"Nearby {selected_priority2}: {row['Nearest_Neighborhood_ID2']}, {nearest_neighborhood_name2}")
            # print(f"Combined Distance: {row['Distance_to_Nearest_Neighborhood_Combined']:.2f} km")
            print()
    else:
        print("No results")

Select your two preferred neighborhood priorities:
1. School
2. Hospital
3. Railway Station
4. Metro Station
Your selected priorities are: School and Metro Station

Top 5 Recommended Buildings:
Building ID: 2, Name: Alag Olive

Nearby School: 3, SHRI INDRADEO SINGH INTERNATIONAL SCHOOL
Nearby Metro Station: 85, Ghatkopar Metro Station

Building ID: 17, Name: Alag Olive

Nearby School: 3, SHRI INDRADEO SINGH INTERNATIONAL SCHOOL
Nearby Metro Station: 85, Ghatkopar Metro Station

Building ID: 19, Name: SKYLINE MAGNUS
Nearby School: 10, D J Doshi Gurukul English Medium High School and Junior College of Commerce
Nearby Metro Station: 86, Ghatkopar Metro Station (East side)


Building ID: 14, Name: SKYLINE MAGNUS
Nearby School: 10, D J Doshi Gurukul English Medium High School and Junior College of Commerce
Nearby Metro Station: 86, Ghatkopar Metro Station (East side)


Building ID: 22, Name: Rajshree 11 East

Nearby School: 22, Sheth Virchand Dhanji Secondary English Medium School

Nearby M

In [4]:
#new 2 inpu
#FINAL ONE FOR MULTIPLE PRIORITIES
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import requests

# Define the get_distance function
def get_distance(coords1, coords2):
    access_token = "pk.eyJ1Ijoic2FuaWthZCIsImEiOiJjbHVrNGdmcjgwZnB0MmpsNXUwdmcyMDR6In0.M4cFaJUCdMnCT0plVTnqVQ"  # Replace with your Mapbox access token
    url = f"https://api.mapbox.com/directions-matrix/v1/mapbox/driving/{coords1[1]},{coords1[0]};{coords2[1]},{coords2[0]}"
    params = {
        'access_token': access_token
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an HTTPError for bad status codes
        data = response.json()
        distance_meters = data['durations'][0][1]  # Distance in meters
        distance_km = distance_meters / 1000  # Convert meters to kilometers
        return distance_km
    except requests.exceptions.RequestException as e:
        print(f"Error occurred during API request: {e}")
        return None
    except (KeyError, IndexError) as e:
        print(f"Invalid data format in the API response: {e}")
        return None

# Load datasets
buildings_data = pd.read_csv("25_buildings.csv")
neighborhood_data = pd.read_csv("neighborhood.csv")

# Select the preferred neighborhood priorities
print("Select your two preferred neighborhood priorities:")
print("1. School")
print("2. Hospital")
print("3. Railway Station")
print("4. Metro Station")

priority_mapping = {
    1: "School",
    2: "Hospital",
    3: "Railway Station",
    4: "Metro Station"
}

priority_input1 = int(input("Enter the number corresponding to your first preferred priority: "))
priority_input2 = int(input("Enter the number corresponding to your second preferred priority: "))

# Validate inputs
if priority_input1 not in priority_mapping or priority_input2 not in priority_mapping or priority_input1 == priority_input2:
    print("Invalid inputs. Please enter two different numbers between 1 and 4.")
else:
    selected_priority1 = priority_mapping[priority_input1]
    selected_priority2 = priority_mapping[priority_input2]
    print("Your selected priorities are:", selected_priority1, "and", selected_priority2)

    # Filtering neighborhood.csv based on selected priorities
    filtered_neighborhood_data1 = neighborhood_data[neighborhood_data['n_type'] == selected_priority1].reset_index(drop=True)
    filtered_neighborhood_data2 = neighborhood_data[neighborhood_data['n_type'] == selected_priority2].reset_index(drop=True)

    # Preprocessing buildings_dataset data
    X_buildings = buildings_data[['Latitude', 'Longitude']]

    # Preprocessing neighborhood.csv for both priorities
    X_neighborhood1 = filtered_neighborhood_data1[['Latitude', 'Longitude']]
    X_neighborhood2 = filtered_neighborhood_data2[['Latitude', 'Longitude']]

    # KNN to find nearest neighbors for both priorities
    knn1 = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(X_neighborhood1)
    _, indices1 = knn1.kneighbors(X_buildings)

    knn2 = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(X_neighborhood2)
    _, indices2 = knn2.kneighbors(X_buildings)

    # Calculate distances using Mapbox API for both priorities
    distances_km = []
    for i in range(len(buildings_data)):
        coords1 = (buildings_data.loc[i, 'Latitude'], buildings_data.loc[i, 'Longitude'])
        coords2_1 = (filtered_neighborhood_data1.loc[indices1[i][0], 'Latitude'], filtered_neighborhood_data1.loc[indices1[i][0], 'Longitude'])
        coords2_2 = (filtered_neighborhood_data2.loc[indices2[i][0], 'Latitude'], filtered_neighborhood_data2.loc[indices2[i][0], 'Longitude'])

        distance_km1 = get_distance(coords1, coords2_1)
        distance_km2 = get_distance(coords1, coords2_2)

        combined_distance = distance_km1 + distance_km2
        distances_km.append(combined_distance)

    # Add nearest neighborhood information and distance to buildings dataset
    buildings_data['Nearest_Neighborhood_ID1'] = filtered_neighborhood_data1.iloc[indices1.flatten()]['n_id'].values
    buildings_data['Nearest_Neighborhood_ID2'] = filtered_neighborhood_data2.iloc[indices2.flatten()]['n_id'].values
    buildings_data['Distance_to_Nearest_Neighborhood_Combined'] = distances_km

    # Sort buildings by combined distance
    sorted_buildings_data = buildings_data.sort_values(by='Distance_to_Nearest_Neighborhood_Combined')

    # Print top 5 recommended buildings
    if len(sorted_buildings_data) > 0:
        print("\nTop 5 Recommended Buildings:")
        for index, row in sorted_buildings_data.head(5).iterrows():
            nearest_neighborhood_name1 = filtered_neighborhood_data1[
                filtered_neighborhood_data1['n_id'] == row['Nearest_Neighborhood_ID1']]['n_name'].iloc[0]
            nearest_neighborhood_name2 = filtered_neighborhood_data2[
                filtered_neighborhood_data2['n_id'] == row['Nearest_Neighborhood_ID2']]['n_name'].iloc[0]
            print(f"Building ID: {row['prop_id']}, Name: {row['prop_name']}")
            print(f"Nearby {selected_priority1}: {row['Nearest_Neighborhood_ID1']}, {nearest_neighborhood_name1}")
            print(f"Nearby {selected_priority2}: {row['Nearest_Neighborhood_ID2']}, {nearest_neighborhood_name2}")
            print(f"Combined Distance: {row['Distance_to_Nearest_Neighborhood_Combined']:.2f} km")
            print()
    else:
        print("No results")

Select your two preferred neighborhood priorities:
1. School
2. Hospital
3. Railway Station
4. Metro Station
Your selected priorities are: School and Hospital

Top 5 Recommended Buildings:
Building ID: 12, Name: Rajshree 11 East

Nearby School: 22, Sheth Virchand Dhanji Secondary English Medium School

Nearby Hospital: 75, Vatsal Nicu And Children's Hospital
Combined Distance: 0.20 km

Building ID: 22, Name: Rajshree 11 East

Nearby School: 22, Sheth Virchand Dhanji Secondary English Medium School

Nearby Hospital: 75, Vatsal Nicu And Children's Hospital
Combined Distance: 0.20 km

Building ID: 25, Name: DRUSHTI EMBASSY
Nearby School: 19, Suyash Vidyalaya
Nearby Hospital: 48, Khus-Hal Hospital

Combined Distance: 0.40 km

Building ID: 11, Name: DRUSHTI EMBASSY
Nearby School: 19, Suyash Vidyalaya
Nearby Hospital: 48, Khus-Hal Hospital

Combined Distance: 0.40 km

Building ID: 6, Name: Shree Balaji Orchid
Nearby School: 19, Suyash Vidyalaya
Nearby Hospital: 48, Khus-Hal Hospital

Combine

In [1]:
# FINAL ONE FOR 2 PRIORITIES

import pandas as pd
from sklearn.neighbors import NearestNeighbors
import requests

# Function to get the distance between two coordinates using Mapbox API
def get_distance(coords1, coords2):
    access_token = "pk.eyJ1Ijoic2FuaWthZCIsImEiOiJjbHVrNGdmcjgwZnB0MmpsNXUwdmcyMDR6In0.M4cFaJUCdMnCT0plVTnqVQ"  # Replace with your Mapbox access token
    url = f"https://api.mapbox.com/directions-matrix/v1/mapbox/driving/{coords1[1]},{coords1[0]};{coords2[1]},{coords2[0]}"
    params = {
        'access_token': access_token
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an HTTPError for bad status codes
        data = response.json()
        distance_meters = data['durations'][0][1]  # Distance in meters
        distance_km = distance_meters / 1000  # Convert meters to kilometers
        return distance_km
    except requests.exceptions.RequestException as e:
        print(f"Error occurred during API request: {e}")
        return None
    except (KeyError, IndexError) as e:
        print(f"Invalid data format in the API response: {e}")
        return None

# Load datasets
buildings_data = pd.read_csv("25_buildings.csv")
neighborhood_data = pd.read_csv("neighborhood.csv")

# Select the preferred neighborhood priorities
print("Select your two preferred neighborhood priorities:")
print("1. School")
print("2. Hospital")
print("3. Railway Station")
print("4. Metro Station")

priority_mapping = {
    1: "School",
    2: "Hospital",
    3: "Railway Station",
    4: "Metro Station"
}

priority_input1 = int(input("Enter the number corresponding to your first preferred priority: "))
priority_input2 = int(input("Enter the number corresponding to your second preferred priority: "))

# Validate inputs
if priority_input1 not in priority_mapping or priority_input2 not in priority_mapping or priority_input1 == priority_input2:
    print("Invalid inputs. Please enter two different numbers between 1 and 4.")
else:
    selected_priority1 = priority_mapping[priority_input1]
    selected_priority2 = priority_mapping[priority_input2]
    print("Your selected priorities are:", selected_priority1, "and", selected_priority2)

    # Filter neighborhood data based on selected priorities
    filtered_neighborhood_data1 = neighborhood_data[neighborhood_data['n_type'] == selected_priority1].reset_index(drop=True)
    filtered_neighborhood_data2 = neighborhood_data[neighborhood_data['n_type'] == selected_priority2].reset_index(drop=True)

    # Preprocess buildings dataset
    X_buildings = buildings_data[['Latitude', 'Longitude']]

    # Preprocess neighborhood data for both priorities
    X_neighborhood1 = filtered_neighborhood_data1[['Latitude', 'Longitude']]
    X_neighborhood2 = filtered_neighborhood_data2[['Latitude', 'Longitude']]

    # KNN to find nearest neighbors for both priorities
    knn1 = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(X_neighborhood1)
    _, indices1 = knn1.kneighbors(X_buildings)

    knn2 = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(X_neighborhood2)
    _, indices2 = knn2.kneighbors(X_buildings)

    # Calculate distances using Mapbox API for both priorities and find combined distance
    distances_km = []
    for i in range(len(buildings_data)):
        coords1 = (buildings_data.loc[i, 'Latitude'], buildings_data.loc[i, 'Longitude'])
        coords2_1 = (filtered_neighborhood_data1.loc[indices1[i][0], 'Latitude'], filtered_neighborhood_data1.loc[indices1[i][0], 'Longitude'])
        coords2_2 = (filtered_neighborhood_data2.loc[indices2[i][0], 'Latitude'], filtered_neighborhood_data2.loc[indices2[i][0], 'Longitude'])

        distance_km1 = get_distance(coords1, coords2_1)
        distance_km2 = get_distance(coords1, coords2_2)

        combined_distance = distance_km1 + distance_km2
        distances_km.append(combined_distance)

    # Add nearest neighborhood information and distance to buildings dataset
    buildings_data['Nearest_Neighborhood_ID1'] = filtered_neighborhood_data1.iloc[indices1.flatten()]['n_id'].values
    buildings_data['Nearest_Neighborhood_ID2'] = filtered_neighborhood_data2.iloc[indices2.flatten()]['n_id'].values
    buildings_data['Distance_to_Nearest_Neighborhood_Combined'] = distances_km

    # Sort buildings by combined distance
    sorted_buildings_data = buildings_data.sort_values(by='Distance_to_Nearest_Neighborhood_Combined')

    # Print top 5 recommended buildings
    recommended_buildings = []

    if len(sorted_buildings_data) > 0:
        for index, row in sorted_buildings_data.head(5).iterrows():
            nearest_neighborhood_name1 = filtered_neighborhood_data1[
                filtered_neighborhood_data1['n_id'] == row['Nearest_Neighborhood_ID1']]['n_name'].iloc[0]
            nearest_neighborhood_name2 = filtered_neighborhood_data2[
                filtered_neighborhood_data2['n_id'] == row['Nearest_Neighborhood_ID2']]['n_name'].iloc[0]
            
            building_info = {
                "Building_ID": row['prop_id'],
                "Building_Name": row['prop_name'],
                "Nearby_" + selected_priority1: {
                    "Neighborhood_ID": row['Nearest_Neighborhood_ID1'],
                    "Neighborhood_Name": nearest_neighborhood_name1
                },
                "Nearby_" + selected_priority2: {
                    "Neighborhood_ID": row['Nearest_Neighborhood_ID2'],
                    "Neighborhood_Name": nearest_neighborhood_name2
                },
                "Combined_Distance_km": row['Distance_to_Nearest_Neighborhood_Combined']
            }
            
            recommended_buildings.append(building_info)
    else:
        recommended_buildings = "No results"

    print(recommended_buildings)


Select your two preferred neighborhood priorities:
1. School
2. Hospital
3. Railway Station
4. Metro Station
Your selected priorities are: School and Hospital
[{'Building_ID': 12, 'Building_Name': 'Rajshree 11 East\n', 'Nearby_School': {'Neighborhood_ID': 22, 'Neighborhood_Name': 'Sheth Virchand Dhanji Secondary English Medium School\r\n'}, 'Nearby_Hospital': {'Neighborhood_ID': 75, 'Neighborhood_Name': "Vatsal Nicu And Children's Hospital\r"}, 'Combined_Distance_km': 0.1995}, {'Building_ID': 22, 'Building_Name': 'Rajshree 11 East\n', 'Nearby_School': {'Neighborhood_ID': 22, 'Neighborhood_Name': 'Sheth Virchand Dhanji Secondary English Medium School\r\n'}, 'Nearby_Hospital': {'Neighborhood_ID': 75, 'Neighborhood_Name': "Vatsal Nicu And Children's Hospital\r"}, 'Combined_Distance_km': 0.1995}, {'Building_ID': 25, 'Building_Name': 'DRUSHTI EMBASSY\r', 'Nearby_School': {'Neighborhood_ID': 19, 'Neighborhood_Name': 'Suyash Vidyalaya\r'}, 'Nearby_Hospital': {'Neighborhood_ID': 48, 'Neighborh